In [1]:
from plotly.subplots import make_subplots
from plotting import plot_values, update_layout
import glob
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
import json
import os
from collections import defaultdict
import matplotlib.pyplot as plt

### Get the names of all files we want to read 

In [14]:
def get_env_name(x):
    x = x.replace("../data_back/", "")
    return x.split("/")[0]

In [15]:
steps = defaultdict(list)
for experiment_file in glob.glob("../data_back/*/*/log.txt", recursive=True):
    task = get_env_name(experiment_file)
    out = pd.read_csv(experiment_file, sep="\t")["timesteps"].max()
    steps[task].append(out)

for task, out in steps.items():
    print(task, min(out), max(out), sum(out)/len(out))

{
    task: max(out)
    for task, out in steps.items()
}

SafetyHalfCheetahVelocity-v1 80000000 240000000 170000000.0
SafetyWalker2dVelocity-v1 2973715 152526534 44358574.625
SafetyHumanoidVelocity-v1 18411743 454764405 140444493.775
SafetyAntVelocity-v1 49415485 159665636 124507630.83333333
SafetyHopperVelocity-v1 561095 58802767 31845059.1875
SafetySwimmerVelocity-v1 20000000 100000000 68888888.8888889


{'SafetyHalfCheetahVelocity-v1': 240000000,
 'SafetyWalker2dVelocity-v1': 152526534,
 'SafetyHumanoidVelocity-v1': 454764405,
 'SafetyAntVelocity-v1': 159665636,
 'SafetyHopperVelocity-v1': 58802767,
 'SafetySwimmerVelocity-v1': 100000000}

In [5]:
for task, outs in steps.items():
    print(task, len(outs))

HalfCheetah 32
Walker2d 24
Humanoid 40
Ant 12
Hopper 16
Swimmer 36


In [ ]:
for algo, data1 in data.groupby("algo"):
    x = data1.drop(["dir_path", 'seed', 'n_iter', 'n_workers', 'reward', "one_sided"], axis=1)
    x = x[['task'] + [col for col in x if col not in ("env_name", "task", "algo")]].sort_values('task', ignore_index=True)
    print(
        x.to_latex(
            index=False,
            caption=f"Optimal hyperparameters from search for {algo}",
            escape=True,
            float_format="{:.2f}".format
        )
        .replace("\\toprule", "\\hline")
        .replace("\\midrule", "\\hline")
        .replace("\\bottomrule", "\\hline")
    )


In [22]:
def plot_env(env_name, data):
    df = []
    for filename, algo in zip(data.dir_path, data.algo):
        x = pd.read_csv("../"+filename+"/log.txt", sep="\t")
        # x["algorithm"] = "ARS-1" if one_sided else "ARS-2"
        x["algorithm"] = algo
        df.append(x)
    df = pd.concat(df, axis=0).groupby(["Iteration", "algorithm"])[["timesteps", "AverageReward"]].mean().reset_index()
    # df.drop("Iteration", axis=1, inplace=True)
    df.rename({
        'timesteps': "steps",
        'AverageReward': "reward"
    }, axis=1, inplace=True)
    df.steps = df.steps.astype(int)
    df["env_name"] = env_name
    fig = px.line(data_frame=df, x="Iteration", y="reward", color="algorithm")
    update_layout(fig, env_name.replace("Safety","").replace("Velocity", ""), "Iteration", "Reward", row=1, col=1, upkwargs=dict(width=500, height=400))
    if "swimmer" in env_name.lower() or "walker" in env_name.lower() or "cheetah" in env_name.lower():
        fig.update_layout(legend=dict(xanchor="right", x=0.99, y=0.5))
    if "human" in env_name.lower():
        fig.update_layout(legend=dict(xanchor="left", x=0.0, y=0.5))
    fig.update_traces(opacity=.7)
    fig.write_image(f"../images/{env_name}.png", scale=3)
    fig.show()
    return df

In [ ]:
for env_name, df in data.groupby("task"):
    plot_env(env_name, df)


### Unstability of SFR-1 wrt hyperparams.

In [ ]:
sfr1 = []
for path, one_sided in zip(all_data.dir_path, all_data.one_sided):
    x = pd.read_csv(os.path.join("..", path, "log.txt"), sep="\t")
    _, x['task'], x['dir_path'] = path.split("/")
    x["one_sided"] = one_sided
    sfr1.append(x)
sfr1 = pd.concat(sfr1)
sfr1.head()

In [ ]:
std1 = sfr1.groupby(["task","Iteration", "one_sided"]).AverageReward.std().reset_index()
std1

In [ ]:
for env_name, df in std1.groupby("task"):
    fig = px.line(data_frame=df, x="Iteration", y="AverageReward", color="one_sided")
    update_layout(fig, env_name.replace("Safety","").replace("Velocity", ""), "Iteration", "reward stddev", row=1, col=1, upkwargs=dict(width=500, height=400))
    fig.show()